In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

### Data Prep 

We have used a synthetic data to demonstrate the implementation. This data preparation can be achieved in SQL in needed.
This example demonstrate the model using two features such as deposit and withdrawal amount.

In [ ]:
data_cm = pd.read_csv('synth_data_aml_cm.csv')
# data_cm.columns

In [ ]:
## selecting fetures related to deposit and withdrawals only as an example
data_cm = data_cm.filter(['Customer_id', 'deposit_amount_24_hours', 'deposit_amount_14_days',
                          'deposit_amount_3_months', 'deposit_amount_lifetime',
                          'n_deposit_days_14_days', 'n_deposit_days_3_months',
                          'n_deposit_days_lifetime', 'wtd_amount_24_hours', 'wtd_amount_14_days', 
                          'wtd_amount_3_months', 'wtd_amount_lifetime', 'n_wtd_days_14_days', 
                          'n_wtd_days_3_months', 'n_wtd_days_lifetime'])

In [ ]:
data_cm.head()

In [ ]:
## data prep stage - can be done via sql 
data_cm['avg_dep_14_days'] = data_cm.deposit_amount_14_days/data_cm.n_deposit_days_14_days
data_cm['avg_dep_3_months'] = data_cm.deposit_amount_3_months /data_cm.n_deposit_days_3_months
data_cm['avg_dep_lifetime'] = data_cm.deposit_amount_lifetime /data_cm.n_deposit_days_lifetime
data_cm['avg_wtd_14_days'] = data_cm.wtd_amount_14_days/data_cm.n_wtd_days_14_days
data_cm['avg_wtd_3_months'] = data_cm.wtd_amount_3_months /data_cm.n_wtd_days_3_months
data_cm['avg_wtd_lifetime'] = data_cm.wtd_amount_lifetime /data_cm.n_wtd_days_lifetime

In [ ]:
data_cm.head()

In [ ]:
data_results = data_cm.set_index(['Customer_id']).filter(regex=('24|avg'))
data_results.head()

In [ ]:
data_results.dtypes

### Model Implementation

In [ ]:
def anomaly_score_feature(df):
    # compute f(x)
    newdf = df.filter(['Customer_id'])
    newdf['flag1'] = df.filter(regex='24').iloc[:,0] > df.filter(regex='14').iloc[:,0]
    newdf['flag2'] = df.filter(regex='24').iloc[:,0] > df.filter(regex='3_months').iloc[:,0]
    newdf['flag3'] = df.filter(regex='24').iloc[:,0] > df.filter(regex='lifetime').iloc[:,0]

    ## computer anomaly score per feature
    newdf = pd.DataFrame(newdf.apply('mean', axis=1))

    return newdf

In [ ]:
## get anomaly score for all available features 
output=[]
feature_list = ['dep', 'wtd']
for value in feature_list:
    df = data_results.filter(regex=value)
    output.append(anomaly_score_feature(df))

In [ ]:
anomaly_score = pd.concat(output, axis=1)
anomaly_score = pd.DataFrame(anomaly_score.apply('mean', axis=1))
anomaly_score.reset_index(level=[0], inplace=True)
anomaly_score.rename(columns={0: 'Anomaly score'}, inplace=True)
anomaly_score.head()